---
## Step 1: Setup and Dependencies

First, get an API key from [Perplexity](https://www.perplexity.ai/account/api/keys). Store this key in a file named `.env`:

```bash
PERPLEXITY_API_KEY=pplx-YOUR_API_KEY
```

---

Then import everything we need. The key components are:

- **`QuestionSet`**: Generates questions from a template and word sets
- **`Workflow`**: Orchestrates asking questions and collecting answers
- **`SQLiteStorageProvider`**: Caches answers in a local database
- **`SonarQueryHandler`**: Sends questions to the Perplexity Sonar API
- **`BaseModel`** (from Pydantic): Defines the structure of expected answers

In [1]:
from pathlib import Path
from typing import Optional
from enum import Enum

from robora import QuestionSet, Workflow, SQLiteStorageProvider
from robora.sonar_query import SonarQueryHandler
from pydantic import BaseModel, Field
import pandas as pd

## 1. Define a Response Model

Pydantic model with `Field(description=...)` to guide the LLM output structure.

In [2]:
class ConfidenceLevel(Enum):
    HIGH = "HIGH"
    MEDIUM = "MEDIUM"
    LOW = "LOW"


class CapitalCityModel(BaseModel):
    country: str = Field(description="The country name.")
    capital_city: str = Field(description="Capital city. Return 'UNKNOWN' if not found.")
    population: Optional[int] = Field(default=None, description="Capital population if available.")
    confidence: ConfidenceLevel = Field(description="Confidence in this answer.")

## 2. Create a QuestionSet

Template with `{placeholder}` + word sets dict → generates all question combinations.

In [3]:
template = "What is the capital city of {country}?"
countries = ["France", "Japan", "Brazil", "Egypt", "Australia"]

question_set = QuestionSet(
    word_sets={"country": countries},
    template=template,
    response_model=CapitalCityModel,
)

# question_set.max_questions = 2  # Limit for testing

## 3. Set Up and Run Workflow

Workflow = QueryHandler + StorageProvider. Results are cached in SQLite.

In [4]:
output_dir = Path("../outputs/tutorial")
output_dir.mkdir(parents=True, exist_ok=True)

storage = SQLiteStorageProvider(str(output_dir / "tutorial.db"))
query_handler = SonarQueryHandler(CapitalCityModel)

workflow = Workflow(
    query_handler=query_handler,
    storage=storage,
    workers=2,
)

In [5]:
# Execute - first run hits API, subsequent runs use cache
answers = await workflow.ask_multiple(question_set, return_results=True)
print(f"Received {len(answers)} answers")

Received 5 answers


## 4. Process Results

In [6]:
if answers:
    results_df = pd.concat([ans.flattened for ans in answers]).reset_index(drop=True)
    results_df.drop(columns=["enriched_citations"], inplace=True, errors='ignore')
    display(results_df)
    
    # Save
    results_df.to_csv(output_dir / "capital_cities.csv", index=False)

,question,error,country,capital_city,population,confidence
0,What is the capital city of Japan?,None,Japan,Tokyo,14000000,ConfidenceLevel.HIGH
1,What is the capital city of Egypt?,None,Egypt,Cairo,10000000,ConfidenceLevel.HIGH
2,What is the capital city of France?,None,France,Paris,2048472,ConfidenceLevel.HIGH
3,What is the capital city of Australia?,None,Australia,Canberra,473855,ConfidenceLevel.HIGH
4,What is the capital city of Brazil?,None,Brazil,Brasília,2817000,ConfidenceLevel.HIGH
